In [130]:
import pandas as pd
from AUIDFunctions import *
import inspect
uh = pd.read_csv("input/unit-history_demo.csv")
am = pd.read_csv("input/all-members_demo.csv")
ac = pd.read_csv("input/accused_demo.csv", dtype={"Middle.Initial":str})
am = am.rename(columns = {'TID' : 'ID1'})
uh = uh.rename(columns = {'TID' : 'ID2'})

In [161]:
pd.set_option('display.max_columns', 30)

In [131]:
def intersect(a, b):
     return list(set(a) & set(b))
def setdiff(a,b):
    return list(set(a) - set(b))
def union(a,b):
    return list(set(a) | set(b))

In [132]:
TakeFirst4 = lambda x: x[:4]
BY_to_CA = lambda x: 2016 - x
def AddColumns(df, add_cols = ["F4FN", "F4LN", "Current.Age", "CA_BY"]):
    if "F4FN" in add_cols and "First.Name" in df.columns:
            df['F4FN'] = df['First.Name'].map(TakeFirst4)
    if "F4LN" in add_cols and 'Last.Name' in df.columns:
            df['F4LN'] = df['Last.Name'].map(TakeFirst4)
    if "Current.Age" in add_cols and "Current.Age" in df.columns:
        df['Current.Age.p1'] = df['Current.Age']
        df['Current.Age.m1'] = df['Current.Age']
    if "CA_BY" in add_cols and "Birth.Year" in df.columns:
        by_to_ca = lambda x: 2016 - x
        df['Current.Age.p1'] = df['Birth.Year'].map(BY_to_CA)
        df['Current.Age.m1'] = df['Birth.Year'].map(BY_to_CA) - 1  
    return df



In [133]:
amd = am.dropna(axis=1, how='all')
uhd = uh.dropna(axis=1, how='all')
amdm = AddColumns(amd)
uhdm = AddColumns(uhd)
print(amd.columns)
print(uhd.columns)
cols = [col for col in amd.columns if col in uhd.columns]
print(cols)

Index(['ID1', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'F4FN', 'F4LN',
       'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
Index(['ID2', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Current.Age', 'Gender', 'Race', 'Star1', 'Star2',
       'Star3', 'Star4', 'Star5', 'Star6', 'Star7', 'Star8', 'Star9', 'Star10',
       'Current.Unit', 'F4FN', 'F4LN', 'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'F4FN', 'F4LN', 'Current.Age.p1', 'Current.Age.m1']


In [134]:
mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t1 = amdmp.merge(uhdmp, on = mc, how ='inner')

mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t2 = amdmp.merge(uhdmp, on = mc, how ='inner')

print(intersect(t1['ID2'], t2['ID2']))
print(intersect(t1['ID1'], t2['ID1']))
print(t1.shape)
print(t2.shape)

[]
[]
(1312, 29)
(4289, 29)


In [250]:
def LoopMerge(df1, df2, on_list, keep_columns = ['ID1', 'ID2'], left_on = [], right_on = [], return_unmerged = True):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    # Generate on list?
    for mc in on_list:
        mc1 = mc + left_on
        mc2 = mc + right_on
        df1t = RemoveDuplicates(df1[keep_columns[:1] + mc1], mc1)
        df2t = RemoveDuplicates(df2[keep_columns[1:] + mc2], mc2)
        dfmt = df1t.merge(df2t, left_on =mc1, right_on=mc2, how='inner')
        if dfmt.shape[0] > 0:
            print('******')
            print(mc)
            print(dfmt.shape[0])
            print('******')
            dfmt['Match'] = '-'.join(mc + left_on + right_on)
            dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
            df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
            df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    if returned_unmerged:
        return (dfm.reset_index(drop=True), df1, df2)
    else: return dfm.reset_index(drop=True)

In [136]:
m = LoopMerge(amdm, uhdm, on_list = [
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
])

******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
4289
******
******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1']
1312
******
******
['F4FN', 'F4LN', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
1
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
19949
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1']
6011
******
******
['F4FN', 'F4LN', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
3
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Middle.Initial', 'Appointed.Date', 'Gender', 'Race']
1
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
2
******
******
['First.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Dat

In [137]:
uid_ref = pd.concat([m[0][["ID1", "ID2"]], m[1][["ID1"]], m[2][["ID2"]]]).reset_index(drop=True)
if "UID" not in uid_ref.columns:
    uid_ref.insert(0, "UID", uid_ref.index + 1)
am.head()

,ID1,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Gender,Race,Birth.Year
0,1,JEFFERY,AARON,M,NaN,2005-09-26,MALE,WHITE,1971
1,2,KARINA,AARON,NaN,NaN,2005-09-26,FEMALE,HISPANIC,1980
2,3,DANIEL,ABATE,P,NaN,1970-06-15,MALE,WHITE,1942
3,4,ANTHONY,ABBATE,G,NaN,1994-12-05,MALE,WHITE,1968
4,5,CARMEL,ABBATE,G,NaN,1969-01-06,MALE,WHITE,1942


In [138]:
uid_ref.head()

,UID,ID1,ID2
0,1,2,2
1,2,16,16
2,3,17,17
3,4,32,31
4,5,33,32


In [139]:
am = am.merge(uid_ref[["ID1", "UID"]], on='ID1', how='left')
uh = uh.merge(uid_ref[["ID2", "UID"]], on='ID2', how='left')

In [140]:
ref = pd.concat([am, uh])
ref.head()

,Appointed.Date,Birth.Year,Current.Age,Current.Unit,First.Name,Gender,ID1,ID2,Last.Name,Middle.Initial,...,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Suffix.Name,UID
0,2005-09-26,1971.0,NaN,NaN,JEFFERY,MALE,1,NaN,AARON,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5603
1,2005-09-26,1980.0,NaN,NaN,KARINA,FEMALE,2,NaN,AARON,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1970-06-15,1942.0,NaN,NaN,DANIEL,MALE,3,NaN,ABATE,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25552
3,1994-12-05,1968.0,NaN,NaN,ANTHONY,MALE,4,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5604
4,1969-01-06,1942.0,NaN,NaN,CARMEL,MALE,5,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5605


In [141]:
ref['Middle.Initial'] = ref['Middle.Initial'].fillna("")
print(ref.columns)
ref.to_csv("test-reference.csv", index=False)

Index(['Appointed.Date', 'Birth.Year', 'Current.Age', 'Current.Unit',
       'First.Name', 'Gender', 'ID1', 'ID2', 'Last.Name', 'Middle.Initial',
       'Race', 'Star1', 'Star10', 'Star2', 'Star3', 'Star4', 'Star5', 'Star6',
       'Star7', 'Star8', 'Star9', 'Suffix.Name', 'UID'],
      dtype='object')


In [142]:
ref.head()

,Appointed.Date,Birth.Year,Current.Age,Current.Unit,First.Name,Gender,ID1,ID2,Last.Name,Middle.Initial,...,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Suffix.Name,UID
0,2005-09-26,1971.0,NaN,NaN,JEFFERY,MALE,1,NaN,AARON,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5603
1,2005-09-26,1980.0,NaN,NaN,KARINA,FEMALE,2,NaN,AARON,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1970-06-15,1942.0,NaN,NaN,DANIEL,MALE,3,NaN,ABATE,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25552
3,1994-12-05,1968.0,NaN,NaN,ANTHONY,MALE,4,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5604
4,1969-01-06,1942.0,NaN,NaN,CARMEL,MALE,5,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5605


In [144]:
'''profiles = AggregateData(ref, uid = "UID", id_cols = [],
              max_cols = setdiff(ref.columns, ["UID"]), make_uid=False)'''

'profiles = AggregateData(ref, uid = "UID", id_cols = [],\n              max_cols = setdiff(ref.columns, ["UID"]), make_uid=False)'

In [145]:
uid = "UID"
uid_col = [uid]
agg_df = ref[uid_col].drop_duplicates().reset_index(drop=True) 
max_cols = setdiff(ref.columns, ["UID"])
print(max_cols)

['Appointed.Date', 'ID2', 'Star7', 'Star4', 'Star9', 'Star2', 'Star5', 'First.Name', 'Last.Name', 'Star1', 'Current.Unit', 'Current.Age', 'ID1', 'Suffix.Name', 'Star3', 'Gender', 'Middle.Initial', 'Star8', 'Birth.Year', 'Star6', 'Race', 'Star10']


In [146]:
profiles = ref[uid_col + max_cols].drop_duplicates()
profiles = profiles.groupby(uid_col, as_index=False)[max_cols]

In [147]:
profiles = profiles.agg(max)

In [148]:
profiles.columns

Index(['UID', 'Appointed.Date', 'ID2', 'Star7', 'Star4', 'Star9', 'Star2',
       'Star5', 'First.Name', 'Last.Name', 'Star1', 'Current.Unit',
       'Current.Age', 'ID1', 'Suffix.Name', 'Star3', 'Gender',
       'Middle.Initial', 'Star8', 'Birth.Year', 'Star6', 'Race', 'Star10'],
      dtype='object')

In [149]:
acd = ac.dropna(axis=1, how='all')
acd = AddColumns(acd)
profiles = AddColumns(profiles)

In [191]:
#def StarMerge(multistar,singlestar, max_star, on_list, keep_columns = ["ID1", "ID2"]):
acdt = acd.loc[acd["Star"]>0]
pt = profiles.loc[(profiles["Star1"].notnull()) & (profiles["Star1"] > 0)]

In [201]:
mc = ['First.Name', 'Last.Name','Appointed.Date', 'Gender', "Race"]
mt = acdt.merge(pt, how = "inner", left_on = mc + ["Star"], right_on= mc + ["Star1"])

In [249]:
keep_columns = ["TID", "UID"]
on_list =  [['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender'],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Middle.Initial", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Middle.Initial", 'Appointed.Date', 'Gender'],
    ['F4FN', 'F4LN', "Middle.Initial", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', "Race", "Current.Unit"],
    ['F4FN', 'F4LN', 'Appointed.Date', 'Gender', "Race", "Current.Unit"],
    ['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name','Appointed.Date', 'Gender', "Birth.Year"],
    ['First.Name','Appointed.Date', 'Gender', "Race","Birth.Year"],
    ['First.Name','Appointed.Date', 'Gender',"Birth.Year"]]

'''dfm = pd.DataFrame(columns = keep_columns + ['Match'])
# Generate on list?
starl = "Star"
df1 = profiles
df2 = acd
print(df1.shape)
print(df2.shape)
total_matches = 0
for mc in on_list:
    for star_left in [col for col in df1.columns if "Star" in col]:
        mc_left = mc + [star_left]
        df1t
        for star_right [col for col in df2.columns if "Star" in col]:
        df2 = profiles.loc[(profiles[starr].notnull()) & (profiles[starr] > 0)]
        print(starr)
        mcr = mc + [starr]
        df1t = RemoveDuplicates(df1[keep_columns[:1] + mcl], mcl)
        df2t = RemoveDuplicates(df2[keep_columns[1:] + mcr], mcr)
        dfmt = df1t.merge(df2t, left_on = mcl, right_on = mcr, how='inner')
        if dfmt.shape[0] > 0:
            total_matches += dfmt.shape[0]
            print('******')
            dfmt['Match'] = '-'.join(mcr)
            dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
            df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
            df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
print(total_matches)'''

'dfm = pd.DataFrame(columns = keep_columns + [\'Match\'])\n# Generate on list?\nstarl = "Star"\ndf1 = profiles\ndf2 = acd\nprint(df1.shape)\nprint(df2.shape)\ntotal_matches = 0\nfor mc in on_list:\n    for star_left in [col for col in df1.columns if "Star" in col]:\n        mc_left = mc + [star_left]\n        df1t\n        for star_right [col for col in df2.columns if "Star" in col]:\n        df2 = profiles.loc[(profiles[starr].notnull()) & (profiles[starr] > 0)]\n        print(starr)\n        mcr = mc + [starr]\n        df1t = RemoveDuplicates(df1[keep_columns[:1] + mcl], mcl)\n        df2t = RemoveDuplicates(df2[keep_columns[1:] + mcr], mcr)\n        dfmt = df1t.merge(df2t, left_on = mcl, right_on = mcr, how=\'inner\')\n        if dfmt.shape[0] > 0:\n            total_matches += dfmt.shape[0]\n            print(\'******\')\n            dfmt[\'Match\'] = \'-\'.join(mcr)\n            dfm = dfm.append(dfmt[keep_columns + [\'Match\']].reset_index(drop=True))\n            df1 = df1.loc[~d

In [257]:
on_cols = on_cols
dfm = pd.DataFrame(columns = keep_columns + ['Match'])
df1 = profiles
df2 = ac
df2_stars = df2.loc[(df2["Star"].notnull()) & (df2["Star"] > 0)]
for mc_cols in on_cols:
    
    dfm = dfm.append(LoopMerge(df1, df2,
                               on_cols = mc_col * 10, keep_columns = ["UID", "TID"],
                               left_on = [], right_on = [], return_unmerged = True)[
        keep_columns + ['Match']])
    df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
    df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]

Star1
Star2
Star3
Star4
Star5
Star6
Star7
Star8
Star9
Star10


In [157]:
'''m = LoopMerge(profiles, acd, on_list = [
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Middle.Initial", 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Middle.Initial", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', "Race", "Current.Unit"],
    ['F4FN', 'F4LN', 'Appointed.Date', 'Gender', "Race", "Current.Unit"],
    ['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name','Appointed.Date', 'Gender', "Birth.Year"],
    ['First.Name','Appointed.Date', 'Gender', "Race","Birth.Year"]
], keep_columns = ["UID", "TID"])
for i in m:
    print(i.shape)'''

******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
2
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
13381
******
******
['F4FN', 'F4LN', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
1
******
******
['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Unit']
15
******
******
['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', 'Race']
1
******
******
['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', 'Birth.Year']
2598
******
******
['First.Name', 'Appointed.Date', 'Gender', 'Race', 'Birth.Year']
7
******
(16005, 3)
(15938, 27)
(17, 16)


In [158]:
m[2].sort_values(["First.Name", "Last.Name"])

,TID,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Birth.Year,Gender,Race,Current.Unit,Current.Rank,Star,F4FN,F4LN,Current.Age.p1,Current.Age.m1
12770,12771,ANGELICA,ARROYO,NaN,NaN,2005-12-05,1971,FEMALE,WHITE,12,PO,14342,ANGE,ARRO,45,44
6746,6747,AUDREY,JURCZYKOWSKI,NaN,NaN,1999-05-10,1970,FEMALE,WHITE,213,PO,8280,AUDR,JURC,46,45
12072,12073,CAROLINE,DEWINTER,NaN,NaN,2003-01-27,1971,FEMALE,WHITE,441,PO,18744,CARO,DEWI,45,44
15180,15181,ERIC,VALDES,NaN,NaN,2016-08-29,1984,MALE,HISPANIC,44,PO,0,ERIC,VALD,32,31
13353,13354,EVELYN,GRUBE,NaN,NaN,2005-09-26,1975,FEMALE,WHITE,9,PO,11676,EVEL,GRUB,41,40
4428,4429,GEORGINA,CUMMINGS,NaN,NaN,1993-11-22,1965,FEMALE,WHITE,22,PO,15449,GEOR,CUMM,51,50
11670,11671,GRACIBEL,HANSEN,NaN,NaN,2004-10-25,1973,FEMALE,WHITE,8,PO,5512,GRAC,HANS,43,42
15031,15032,JAMEL,JUDEH,NaN,NaN,2012-10-05,1983,MALE,HISPANIC,44,PO,-999,JAME,JUDE,33,32
14570,14571,JAMES,BANSLEY,NaN,NaN,2009-12-16,1983,MALE,WHITE,193,PO,10927,JAME,BANS,33,32
4275,4276,JOSE,CHAMBERS,NaN,NaN,2000-01-24,1972,MALE,BLACK,-999,PO,-999,JOSE,CHAM,44,43


In [162]:
print(m[1].loc[m[1]["First.Name"]=="AUDREY"])

         UID Appointed.Date      ID2  Star7  Star4  Star9  Star2  Star5  \
1819    1820     1999-05-10  13864.0 -999.0 -999.0 -999.0 -999.0 -999.0   
5081    5082     1998-10-26  19437.0 -999.0 -999.0 -999.0 -999.0 -999.0   
7078    7079     1985-05-06   2320.0 -999.0 -999.0 -999.0 -999.0 -999.0   
10740  10741     1993-12-13   8034.0 -999.0 -999.0 -999.0 -999.0 -999.0   
20957  20958     1986-10-13  24181.0 -999.0 -999.0 -999.0 -999.0 -999.0   
29537  29538     1986-10-13  20588.0 -999.0 -999.0 -999.0 -999.0 -999.0   
31745  31746     1999-05-10      NaN    NaN    NaN    NaN    NaN    NaN   

      First.Name     Last.Name    Star1  Current.Unit  Current.Age      ID1  \
1819      AUDREY  JURCZYKOWSKI   8280.0          20.0         45.0  14018.0   
5081      AUDREY       MORALES  16997.0          10.0         44.0  19643.0   
7078      AUDREY          BONE  17566.0          55.0         58.0   2348.0   
10740     AUDREY         ESTES  12238.0           3.0         53.0   8126.0   
2095